In [34]:
import torch
from opt_nets import MyNetwork, get_predictions
import numpy as np
from submission_utils import get_submission_markowitz, general_weights_fixer, test_submission
from data_utils import read_all_candles
import pandas as pd


In [3]:
network = MyNetwork(
        96,
        max_weight=0.2,
        force_symmetric=True,
        n_clusters=2,
        n_init=100,
        init="k-means++",
        random_state=None,
    )

In [5]:
network.load_state_dict(torch.load("models_2403/mynetwork_2403.pt"))

<All keys matched successfully>

In [7]:
def get_predictions(model, n_channels, lookback, n_assets):
    w_pred = (
        model
        .to("cpu")(torch.ones(1, n_channels, lookback, n_assets).cpu())
        .cpu()
        .detach()
        .numpy()
        .squeeze()
    )
    return w_pred

In [8]:
preds = get_predictions(network, 4, 60, 96)

In [9]:
preds

array([-0.25765452, -0.33364168,  0.68374044, -0.01000493,  0.05360133,
       -0.04072409,  0.03489765, -0.03997862,  0.67171174, -0.5362123 ,
        0.35966536,  0.2660258 , -0.6722335 ,  0.8437172 , -0.02823243,
       -0.8957228 , -0.09132884, -0.39747235,  0.15730008,  0.09125657,
        0.26792106, -0.11328404, -0.1304497 , -0.01313755, -0.26236245,
        0.05742706, -0.23120216, -0.1857794 , -0.09724005,  0.20235777,
       -0.08224139, -0.25102645, -0.4722216 , -0.4686712 ,  0.549304  ,
        0.21912408,  0.10653103,  0.06932335, -0.1372239 ,  0.12150095,
        0.2904611 ,  0.19688725, -2.360684  ,  0.41441184,  0.08262783,
        0.09113218,  0.30425635, -0.25680572, -0.13463981, -0.11757946,
        0.57206404, -0.08529983,  0.44372186,  0.17515022,  0.5770836 ,
       -0.2641881 ,  0.2625934 , -0.07316159,  0.02884555,  0.2958549 ,
       -0.06818055, -0.19900487,  0.06335143, -0.11180976,  0.4057709 ,
        0.1618715 ,  0.18365976,  0.34680125, -0.14527431,  0.07

In [11]:
preds = np.clip(preds, 0, 0.2)

In [12]:
preds = preds / preds.sum()

In [13]:
preds.sum()

1.0

In [14]:
preds

array([0.        , 0.        , 0.02474024, 0.        , 0.00663055,
       0.        , 0.00431688, 0.        , 0.02474024, 0.        ,
       0.02474024, 0.02474024, 0.        , 0.02474024, 0.        ,
       0.        , 0.        , 0.        , 0.01945821, 0.01128855,
       0.02474024, 0.        , 0.        , 0.        , 0.        ,
       0.0071038 , 0.        , 0.        , 0.        , 0.02474024,
       0.        , 0.        , 0.        , 0.        , 0.02474024,
       0.02474024, 0.01317802, 0.00857538, 0.        , 0.01502981,
       0.02474024, 0.02435519, 0.        , 0.02474024, 0.01022116,
       0.01127316, 0.02474024, 0.        , 0.        , 0.        ,
       0.02474024, 0.        , 0.02474024, 0.02166629, 0.02474024,
       0.        , 0.02474024, 0.        , 0.00356823, 0.02474024,
       0.        , 0.        , 0.00783665, 0.        , 0.02474024,
       0.0200237 , 0.02271893, 0.02474024, 0.        , 0.00912912,
       0.        , 0.02474024, 0.02474024, 0.02474024, 0.02474

In [17]:
candles = read_all_candles()

Getting candles data for Darwins...: 100%|██████████| 96/96 [00:01<00:00, 64.74it/s]


In [19]:
submal = pd.read_csv("submission_2103.csv")

In [20]:
submal.head()

,eod_ts,allo_ZVQ,allo_NCT,allo_YAX,allo_OOS,allo_GFJ,allo_FIR,allo_USX,allo_FSK,allo_TMF,...,allo_AZG,allo_OJG,allo_WWT,allo_BOT,allo_TXR,allo_RAT,allo_DIG,allo_SRI,allo_PHI,allo_RWJ
0,2020-08-18 00:00:00,0.0,0.0,0.7653,0.0,0.0571,0.0,0.034,0.0,0.7199,...,0.0,0.8757,0.0,0.0,0.2181,0.2732,0.0,0.3957,0.0048,0.4489
1,2020-08-18 01:00:00,0.0,0.0,0.7653,0.0,0.0571,0.0,0.034,0.0,0.7199,...,0.0,0.8757,0.0,0.0,0.2181,0.2732,0.0,0.3957,0.0048,0.4489
2,2020-08-18 02:00:00,0.0,0.0,0.7653,0.0,0.0571,0.0,0.034,0.0,0.7199,...,0.0,0.8757,0.0,0.0,0.2181,0.2732,0.0,0.3957,0.0048,0.4489
3,2020-08-18 03:00:00,0.0,0.0,0.7653,0.0,0.0571,0.0,0.034,0.0,0.7199,...,0.0,0.8757,0.0,0.0,0.2181,0.2732,0.0,0.3957,0.0048,0.4489
4,2020-08-18 04:00:00,0.0,0.0,0.7653,0.0,0.0571,0.0,0.034,0.0,0.7199,...,0.0,0.8757,0.0,0.0,0.2181,0.2732,0.0,0.3957,0.0048,0.4489


In [23]:
asset_names = [col.replace("allo_", "") for col in submal.columns if col != "eod_ts"]

In [24]:
len(asset_names)

96

In [30]:
preds = general_weights_fixer(preds)

In [31]:
weightsdict = {asset:w for asset, w in zip(asset_names, preds)}

In [32]:
sum(weightsdict.values())

1.0

In [33]:
submission = get_submission_markowitz(weightsdict, asset_names)

In [35]:
test_submission(submission)

In [36]:
submission.head()

,allo_ZVQ,allo_NCT,allo_YAX,allo_OOS,allo_GFJ,allo_FIR,allo_USX,allo_FSK,allo_TMF,allo_TDD,...,allo_AZG,allo_OJG,allo_WWT,allo_BOT,allo_TXR,allo_RAT,allo_DIG,allo_SRI,allo_PHI,allo_RWJ
eod_ts,,,,,,,,,,,,,,,,,,,,,
2020-08-18 00:00:00,0.0,0.0,0.0259,0.0,0.0066,0.0,0.0043,0.0,0.0247,0.0,...,0.0,0.0247,0.0,0.0,0.0247,0.0247,0.0,0.0247,0.0006,0.0247
2020-08-18 01:00:00,0.0,0.0,0.0259,0.0,0.0066,0.0,0.0043,0.0,0.0247,0.0,...,0.0,0.0247,0.0,0.0,0.0247,0.0247,0.0,0.0247,0.0006,0.0247
2020-08-18 02:00:00,0.0,0.0,0.0259,0.0,0.0066,0.0,0.0043,0.0,0.0247,0.0,...,0.0,0.0247,0.0,0.0,0.0247,0.0247,0.0,0.0247,0.0006,0.0247
2020-08-18 03:00:00,0.0,0.0,0.0259,0.0,0.0066,0.0,0.0043,0.0,0.0247,0.0,...,0.0,0.0247,0.0,0.0,0.0247,0.0247,0.0,0.0247,0.0006,0.0247
2020-08-18 04:00:00,0.0,0.0,0.0259,0.0,0.0066,0.0,0.0043,0.0,0.0247,0.0,...,0.0,0.0247,0.0,0.0,0.0247,0.0247,0.0,0.0247,0.0006,0.0247


In [37]:
submission.to_csv("submission_2403.csv", header=True, index=True)